# Build a RAG agent with LangChain

## Overview

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q\&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or [RAG](https://docs.langchain.com/oss/python/langchain/retrieval/).

This tutorial will show how to build a simple Q\&A application over an unstructured text data source. We will demonstrate:

1. A RAG [agent](#rag-agents) that executes searches with a simple tool. This is a good general-purpose implementation.
2. A two-step RAG [chain](#rag-chains) that uses just a single LLM call per query. This is a fast and effective method for simple queries.

### Concepts

We will cover the following concepts:

* **Indexing**: a pipeline for ingesting data from a source and indexing it. *This usually happens in a separate process.*

* **Retrieval and generation**: the actual RAG process, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

Once we've indexed our data, we will use an [agent](https://docs.langchain.com/oss/python/langchain/agents) as our orchestration framework to implement the retrieval and generation steps.

> The indexing portion of this tutorial will largely follow the [semantic search tutorial](https://docs.langchain.com/oss/python/langchain/knowledge-base).
> 
> If your data is already available for search (i.e., you have a function to execute a search), or you're comfortable with the content from that tutorial, feel free to skip to the section on [retrieval and generation](#2-retrieval-and-generation)

### Preview

In this guide we'll build an app that answers questions about the website's content. The specific website we will use is the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng, which allows us to ask questions about the contents of the post.

We can create a simple indexing pipeline and RAG chain to do this in \~40 lines of code. See below for the full code snippet:

In [4]:
import bs4
from langchain.agents import AgentState, create_agent
from langchain_community.document_loaders import WebBaseLoader
from langchain.messages import MessageLikeRepresentation
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [6]:
# Construct a tool for retrieving context
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [10]:
tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Must use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [11]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_0908786027974354b684c2)
 Call ID: call_0908786027974354b684c2
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate inte

## Setup

### Installation

This tutorial requires these langchain dependencies:


In [ ]:
%uv pip install langchain-community~=0.4 langchain-chroma~=1.0 langchain[openai]~=1.0 beautifulsoup4~=4.14 python-dotenv~=1.2

In [ ]:
# 安装 CPU 版 PyTorch，避免 sentence-transformers 自动安装 GPU 版 PyTorch
%uv pip install torch~=2.9 --index-url https://download.pytorch.org/whl/cpu
%uv pip install langchain-huggingface~=1.2 sentence-transformers~=5.2

For more details, see our [Installation guide](https://docs.langchain.com/oss/python/langchain/install).

### Components
We will need to select three components from LangChain’s suite of integrations.
Select a chat model:

In [14]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

model = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE_URL"],
    model=os.environ["OPENAI_MODEL"],
)


Select an embeddings model:

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name='sentence-transformers/all-mpnet-base-v2'
# model_name='BAAI/bge-small-en-v1.5'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Select a vector store:

In [13]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./_rag-agent-db",  # Where to save data locally, remove if not necessary
)

## 1. Indexing

> **This section is an abbreviated version of the content in the [semantic search tutorial](https://docs.langchain.com/oss/python/langchain/knowledge-base).**
> 
> If your data is already indexed and available for search (i.e., you have a function to execute a search), or if you're comfortable with [document loaders](https://docs.langchain.com/oss/python/langchain/retrieval#document_loaders), [embeddings](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models), and [vector stores](https://docs.langchain.com/oss/python/langchain/retrieval#vectorstores), feel free to skip to the next section on [retrieval and generation](https://docs.langchain.com/oss/python/langchain/rag#2-retrieval-and-generation).

Indexing commonly works as follows:

1. **Load**: First we need to load our data. This is done with [Document Loaders](https://docs.langchain.com/oss/python/langchain/retrieval#document_loaders).
2. **Split**: [Text splitters](https://docs.langchain.com/oss/python/langchain/retrieval#text_splitters) break large `Documents` into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window.
3. **Store**: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a [VectorStore](https://docs.langchain.com/oss/python/langchain/retrieval#vectorstores) and [Embeddings](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models) model.

<img src="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=21403ce0d0c772da84dcc5b75cff4451" alt="index_diagram" data-og-width="2583" width="2583" data-og-height="1299" height="1299" data-path="images/rag_indexing.png" data-optimize="true" data-opv="3" srcset="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=280&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=bf4eb8255b82a809dbbd2bc2a96d2ed7 280w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=560&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=4ebc538b2c4765b609f416025e4dbbda 560w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779 840w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=1100&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=675f55e100bab5e2904d27db01775ccc 1100w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=1650&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=4b9e544a7a3ec168651558bce854eb60 1650w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=2500&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=f5aeaaaea103128f374c03b05a317263 2500w" />

### Loading documents

We need to first load the blog post contents. We can use [DocumentLoaders](https://docs.langchain.com/oss/python/langchain/retrieval#document_loaders) for this, which are objects that load in data from a source and return a list of [Document](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects.

In this case we'll use the [`WebBaseLoader`](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base), which uses `urllib` to load HTML from web URLs and `BeautifulSoup` to parse it to text. We can customize the HTML -> text parsing by passing in parameters into the `BeautifulSoup` parser via `bs_kwargs` (see [BeautifulSoup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we'll remove all others.

In [15]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [16]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


**Go deeper**

`DocumentLoader`: Object that loads data from a source as list of `Documents`.

* [Integrations](https://docs.langchain.com/oss/python/integrations/document_loaders/): 160+ integrations to choose from.
* [`BaseLoader`](https://reference.langchain.com/python/langchain_core/document_loaders/#langchain_core.document_loaders.BaseLoader): API reference for the base interface.

### Splitting documents

Our loaded document is over 42k characters which is too long to fit into the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we'll split the [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) into chunks for embedding and vector storage. This should help us retrieve only the most relevant parts of the blog post at run time.

As in the [semantic search tutorial](https://docs.langchain.com/oss/python/langchain/knowledge-base), we use a `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


**Go deeper**

`TextSplitter`: Object that splits a list of [`Document`](https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document) objects into smaller
chunks for storage and retrieval.

* [Integrations](https://docs.langchain.com/oss/python/integrations/splitters/)
* [Interface](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html): API reference for the base interface.

### Storing documents

Now we need to index our 66 text chunks so that we can search over them at runtime. Following the [semantic search tutorial](https://docs.langchain.com/oss/python/langchain/knowledge-base), our approach is to [embed](https://docs.langchain.com/oss/python/langchain/retrieval#embedding_models/) the contents of each document split and insert these embeddings into a [vector store](https://docs.langchain.com/oss/python/langchain/retrieval#vectorstores/). Given an input query, we can then use vector search to retrieve relevant documents.

We can embed and store all of our document splits in a single command using the vector store and embeddings model selected at the [start of the tutorial](https://docs.langchain.com/oss/python/langchain/rag#components).

In [18]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['d9c1174e-8fc7-4448-b8b8-916ef7af93a7', '2ef5b287-eba7-4042-b852-acbc69a8b23f', '87057d97-95f2-4529-9952-ab5fde00cfcc']


**Go deeper**

`Embeddings`: Wrapper around a text embedding model, used for converting text to embeddings.

* [Integrations](https://docs.langchain.com/oss/python/integrations/text_embedding/): 30+ integrations to choose from.
* [Interface](https://reference.langchain.com/python/langchain_core/embeddings/#langchain_core.embeddings.embeddings.Embeddings): API reference for the base interface.

`VectorStore`: Wrapper around a vector database, used for storing and querying embeddings.

* [Integrations](https://docs.langchain.com/oss/python/integrations/vectorstores/): 40+ integrations to choose from.
* [Interface](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html): API reference for the base interface.

This completes the **Indexing** portion of the pipeline. At this point we have a query-able vector store containing the chunked contents of our blog post. Given a user question, we should ideally be able to return the snippets of the blog post that answer the question.

## 2. Retrieval and Generation

RAG applications commonly work as follows:

1. **Retrieve**: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://docs.langchain.com/oss/python/langchain/retrieval#retrievers).
2. **Generate**: A [model](https://docs.langchain.com/oss/python/langchain/models) produces an answer using a prompt that includes both the question with the retrieved data

<img src="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=994c3585cece93c80873d369960afd44" alt="retrieval_diagram" data-og-width="2532" width="2532" data-og-height="1299" height="1299" data-path="images/rag_retrieval_generation.png" data-optimize="true" data-opv="3" srcset="https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=280&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=3bd28b3662e08c8364b60b74f510751e 280w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=560&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=43484903ca631a47a54e86191eb5ba22 560w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=67fe2302e241fc24238a5df1cf56573d 840w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=1100&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=d390a6a758e688ec36352d30b22249b0 1100w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=1650&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=59729377317a0631598b6a4a2a7d8c92 1650w, https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_retrieval_generation.png?w=2500&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=c07711c71153c3b2dfd5b0104ad3e324 2500w" />

Now let's write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

We will demonstrate:

1. A RAG [agent](#rag-agents) that executes searches with a simple tool. This is a good general-purpose implementation.
2. A two-step RAG [chain](#rag-chains) that uses just a single LLM call per query. This is a fast and effective method for simple queries.

### RAG agents

One formulation of a RAG application is as a simple [agent](https://docs.langchain.com/oss/python/langchain/agents) with a tool that retrieves information. We can assemble a minimal RAG agent by implementing a [tool](https://docs.langchain.com/oss/python/langchain/tools) that wraps our vector store:

In [19]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

> Here we use the [tool decorator](https://reference.langchain.com/python/langchain/tools/#langchain.tools.tool) to configure the tool to attach raw documents as [artifacts](https://docs.langchain.com/oss/python/langchain/messages#param-artifact) to each [ToolMessage](https://docs.langchain.com/oss/python/langchain/messages#tool-message). This will let us access document metadata in our application, separate from the stringified representation that is sent to the model.

>  Retrieval tools are not limited to a single string `query` argument, as in the above example. You can
>  force the LLM to specify additional search parameters by adding arguments— for example, a category:
>
>  ```python  theme={null}
>  from typing import Literal
>
>  def retrieve_context(query: str, section: Literal["beginning", "middle", "end"]):
>  ```

Given our tool, we can construct the agent:

In [20]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

Let's test this out. We construct a question that would typically require an iterative sequence of retrieval steps to answer:

In [22]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_f330fa8b956746d4acebc8)
 Call ID: call_f330fa8b956746d4acebc8
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classic

Note that the agent:

1. Generates a query to search for a standard method for task decomposition;
2. Receiving the answer, generates a second query to search for common extensions of it;
3. Having received all necessary context, answers the question.

We can see the full sequence of steps, along with latency and other metadata, in the [LangSmith trace](https://smith.langchain.com/public/7b42d478-33d2-4631-90a4-7cb731681e88/r).

> You can add a deeper level of control and customization using the [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview) framework directly— for example, you can add steps to grade document relevance and rewrite search queries. Check out LangGraph's [Agentic RAG tutorial](https://docs.langchain.com/oss/python/langgraph/agentic-rag) for more advanced formulations.

### RAG chains

In the above [agentic RAG](#rag-agents) formulation we allow the LLM to use its discretion in generating a [tool call](https://docs.langchain.com/oss/python/langchain/models#tool-calling) to help answer user queries. This is a good general-purpose solution, but comes with some trade-offs:

| ✅ Benefits                                                                                                                                                 | ⚠️ Drawbacks                                                                                                                                |
| ---------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **Search only when needed** – The LLM can handle greetings, follow-ups, and simple queries without triggering unnecessary searches.                        | **Two inference calls** – When a search is performed, it requires one call to generate the query and another to produce the final response. |
| **Contextual search queries** – By treating search as a tool with a `query` input, the LLM crafts its own queries that incorporate conversational context. | **Reduced control** – The LLM may skip searches when they are actually needed, or issue extra searches when unnecessary.                    |
| **Multiple searches allowed** – The LLM can execute several searches in support of a single user query.                                                    |                                                                                                                                             |

Another common approach is a two-step chain, in which we always run a search (potentially using the raw user query) and incorporate the result as context for a single LLM query. This results in a single inference call per query, buying reduced latency at the expense of flexibility.

In this approach we no longer call the model in a loop, but instead make a single pass.

We can implement this chain by removing tools from the agent and instead incorporating the retrieval step into a custom prompt:

In [28]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

Let's try this out:

In [29]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This enables better planning, reasoning, and execution—especially for AI agents or humans tackling multi-step problems.

In AI systems, task decomposition can be achieved in several ways:

1. **LLM-based decomposition via prompting**:  
   - Simple prompts like *“Steps for XYZ.\n1.”* or *“What are the subgoals for achieving XYZ?”* guide the LLM to generate an ordered list of subtasks.  
   - More advanced techniques include **Chain-of-Thought (CoT)** (Wei et al., 2022), where the model is prompted to “think step by step”, explicitly revealing its reasoning path; and **Tree-of-Thoughts (ToT)** (Yao et al., 2023), which explores *multiple possible reasoning paths* at each step—forming a

In the [LangSmith trace](https://smith.langchain.com/public/0322904b-bc4c-4433-a568-54c6b31bbef4/r/9ef1c23e-380e-46bf-94b3-d8bb33df440c) we can see the retrieved context incorporated into the model prompt.

This is a fast and effective method for simple queries in constrained settings, when we typically do want to run user queries through semantic search to pull additional context.

**Returning source documents**
The above [RAG chain](#rag-chains) incorporates retrieved context into a single system message for that run.

As in the [agentic RAG](#rag-agents) formulation, we sometimes want to include raw source documents in the application state to have access to document metadata. We can do this for the two-step chain case by:

1. Adding a key to the state to store the retrieved documents
2. Adding a new node via a [pre-model hook](https://docs.langchain.com/oss/python/langchain/agents#pre-model-hook) to populate that key (as well as inject the context).

In [ ]:
from typing import Any
from langchain_core.documents import Document
from langchain.agents.middleware import AgentMiddleware, AgentState


class State(AgentState):
    context: list[Document]


class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
    state_schema = State

    def before_model(self, state: AgentState) -> dict[str, Any] | None:
        last_message = state["messages"][-1]
        retrieved_docs = vector_store.similarity_search(last_message.text)

        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        augmented_message_content = (
            f"{last_message.text}\n\n"
            "Use the following context to answer the query:\n"
            f"{docs_content}"
        )
        return {
            "messages": [last_message.model_copy(update={"content": augmented_message_content})],
            "context": retrieved_docs,
        }


agent = create_agent(
    model,
    tools=[],
    middleware=[RetrieveDocumentsMiddleware()],
)


## Next steps

Now that we've implemented a simple RAG application via [`create_agent`](https://reference.langchain.com/python/langchain/agents/#langchain.agents.create_agent), we can easily incorporate new features and go deeper:

* [Stream](/oss/python/langchain/streaming) tokens and other information for responsive user experiences
* Add [conversational memory](/oss/python/langchain/short-term-memory) to support multi-turn interactions
* Add [long-term memory](/oss/python/langchain/long-term-memory) to support memory across conversational threads
* Add [structured responses](/oss/python/langchain/structured-output)
* Deploy your application with [LangSmith Deployment](/langsmith/deployments)
